# LeNet 5 (A close approximation) on MNIST

### Imports 

In [1]:
# The Keras imports
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AveragePooling2D
from keras.optimizers import SGD
from keras import backend as K
import numpy as np
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


### Defining the basic constants

In [ ]:
classes      = 10        # There are 10 digits
mini_batch   = 128       # The mini-batch size
epochs       = 20       # number of epochs to run (be careful!)
activation   = 'relu'    # pick the activation (relu, tanh)

### Loading the data

In [2]:
# The train and test data splits
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#First, unflatten the images to be 28x28 pixels which they originally are
x_train = x_train.reshape(x_train.shape[0], 28, 28)
x_test = x_test.reshape(x_test.shape[0], 28, 28)

# Add the \"channel\" dimension. For images it is usually the rgb. Here it is grayscale, so we have only 1-channel added
x_train= x_train[:, :, :, np.newaxis]
x_test = x_test[:,:,:, np.newaxis]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Let us scale the values to be between [0,1]. Since the original pixel values are from 0-255\n", we scale it down
x_train /= 255.0
x_test /= 255.0

# The one-hot encoding of the response (0-9 digits). In other words, \"4\" looks like [0,0,0,0,1,0,0,0,0,0]
y_train_hot = to_categorical(y_train, 10)
y_test_hot  = to_categorical(y_test, 10)

# Pad the 28x28 images with 2-pixel border all around, get 32x32. The original paper uses 32x32 size images, so if we have to pass the MNIST images, we need to pad it

# The padding is as follows:
# no padding of the rows, so the first (0,0)
# pad the width in the left and right by 2 pixels, so the (2,2)
# pad the height in the top and bottom by 2 pixels, so the next (2,2)
# no padding of the channels -- we leave it as 1, so the final (0,0)

x_train = np.pad(x_train, ((0,0), (2,2), (2,2), (0,0)), 'constant')
x_test = np.pad(x_test, ((0,0), (2,2), (2,2), (0,0)), 'constant')

### Building the LeNet5 CNN Architecture (Approximately)

In [3]:
def build_LeNet5Model ():
    #An approximation to the original LeNet5 model
    model = Sequential()
    
    # Now, lets build the first Convulational layer with the three pieces: Conv-filter layer, ReLu activation layer, followed by AveragePooling layer
    # Note that today we would instead use MaxPooling, but in those days AveragePooling was customary,
    # Originally, tanh was used as the activation function, though today, it would make sense to use ReLu (much faster!)
    
    conv1 = Conv2D(filters=6,             # Look for six features
                   kernel_size=5,         # Use a 5x5 filter (kernel)
                   strides=1,             # the filter-stride while scanning
                   activation=activation, # activation function
                   input_shape=(32,32,1))  # Gray-scale image of 32x32 pixels
    avePool1 = AveragePooling2D(pool_size=2, strides=2)
    
    conv2 = Conv2D(filters=16,            # Look for sixteen features
                   kernel_size=5,         # Use a 5x5 filter (kernel)
                   strides=1,             # the filter-stride while scanning
                   activation=activation) # activation function
    avePool2 = AveragePooling2D(pool_size=2, strides=2)

    flatten = Flatten()
    fcLayer3     = Dense(units=120, activation=activation)
    fcLayer4     = Dense(units=84,  activation=activation)

    # Finally, the output layer using softmax to recognize the 10 digits
    outputLayer = Dense (10, activation='softmax')

    model.add(conv1)       # first convulation
    model.add(avePool1)    # pooling
    model.add(conv2)       # second convulation
    model.add(avePool2)    # pooling
    model.add(flatten)     # flatten in a single vector
    model.add(fcLayer3)    # fully connected layer of 120 neurons
    model.add(fcLayer4)    # fully connected layer of 84 neurons
    model.add(outputLayer) # output
    return model

### Building and running the model

In [6]:
model = build_LeNet5Model()

model.compile(optimizer=SGD(), # use the SGD (now,Adam would work much faster)
              loss='categorical_crossentropy',
              metrics= ['accuracy'])
                   
# Train the model on the training images
model.fit(x_train,                      # the predictors
          y_train_hot,                  # the response
          batch_size=mini_batch,        # mini-batch size
          epochs = epochs,              # number of epochs to run
          verbose = 1)                  # print out the executions

Epoch 1/20
60000/60000 [==============================] - 15s 256us/step - loss: 1.9886 - acc: 0.3429
Epoch 2/20
60000/60000 [==============================] - 15s 248us/step - loss: 0.4688 - acc: 0.8635
Epoch 3/20
60000/60000 [==============================] - 15s 249us/step - loss: 0.3247 - acc: 0.9023
Epoch 4/20
60000/60000 [==============================] - 15s 250us/step - loss: 0.2707 - acc: 0.9186
Epoch 5/20
60000/60000 [==============================] - 15s 249us/step - loss: 0.2331 - acc: 0.9303
Epoch 6/20
60000/60000 [==============================] - 15s 251us/step - loss: 0.2079 - acc: 0.9375
Epoch 7/20
60000/60000 [==============================] - 15s 250us/step - loss: 0.1843 - acc: 0.9452
Epoch 8/20
60000/60000 [==============================] - 15s 251us/step - loss: 0.1678 - acc: 0.9489
Epoch 9/20
60000/60000 [==============================] - 15s 254us/step - loss: 0.1539 - acc: 0.9533
Epoch 10/20
60000/60000 [==============================] - 15s 252us/step - loss: 

### Evaluating the model for accuracy

In [8]:
# Now evaluate the model for accuracy
(loss, accuracy) = model.evaluate(x_test, 
                                  y_test_hot,
                                  batch_size = mini_batch,
                                  verbose = 1)
print(f'The Accuracy is: {accuracy}, the residual loss is: {loss}')

10000/10000 [==============================] - 1s 96us/step
The Accuracy is: 0.9728, the residual loss is: 0.08115393745303154
